In [1]:
import numpy as np
import pandas as pd

In [2]:
df_soap = pd.read_pickle("df_soap_reduced.pkl")

In [3]:
def extract_importances(output):
    overall_cm = {}
    for idx,estimator in enumerate(output['estimator']):
        feature_importances = pd.DataFrame(estimator.feature_importances_,
                                           index = estimator.feature_names_in_,
                                            columns=['importance']).sort_values('importance', ascending=False)
        feature_importances = feature_importances.rename(columns = {"importance":"importance" + str(idx)})
        overall_cm[idx] = feature_importances


    overall_cm_df = overall_cm[0]
    overall_cm_df = overall_cm_df.merge(overall_cm[1], left_index = True, how = 'outer', right_index =True)
    overall_cm_df = overall_cm_df.merge(overall_cm[2], left_index = True, how = 'outer', right_index =True)
    overall_cm_df = overall_cm_df.merge(overall_cm[3], left_index = True, how = 'outer', right_index =True)
    overall_cm_df = overall_cm_df.merge(overall_cm[4], left_index = True, how = 'outer', right_index =True)
    overall_cm_df["average"] = overall_cm_df.mean(axis = 1)
    return overall_cm_df

def iterative_remove(df_imp, df_data):
    df_reduced = df_data.copy()
    df_imp = df_imp.sort_values("average")
    idx = round(len(df_imp)*0.10)
    df_imp = df_imp[idx:]
    to_keep = df_imp.index.values.tolist()
    df_reduced = df_data[to_keep]
    return df_reduced
def iterative_train(reduced,kf,thresh,params):
    importances_dict = {}
    outputs = {}
    scores = {}
    dfs = {}
    
    while reduced.shape[1] > thresh:
        print(reduced.shape[1])
        
        Xtrains = []
        Xtests = []
        ytrains = []
        ytests = []
        for train_index, test_index in kf.split(reduced):
            Xtrains.append(reduced.iloc[train_index])
            ytrains.append(y.iloc[train_index])
            Xtests.append(reduced.iloc[test_index])
            ytests.append(y.iloc[test_index])
        score_carry = []
        eval_set=[(Xtests[0],ytests[0])]
        xgbr = XGBRegressor(**params, n_estimators = 350)
        xgbr_out = xgbr.fit(Xtrains[0], ytrains[0], 
                    eval_metric='rmse',
                    eval_set = eval_set,
                    verbose=False)
        importances = pd.DataFrame(zip(xgbr_out.feature_importances_, xgbr_out.get_booster().feature_names))
        score_carry.append(list(xgbr_out.evals_result()["validation_0"].values())[-1][-1])
        for i in range(1,5):
            eval_set_carry=[(Xtests[i],ytests[i])]
            xgbr = XGBRegressor(**params, n_estimators = 350)
            xgbr_out_carry = xgbr.fit(Xtrains[i], ytrains[i], 
                    eval_metric='rmse',
                    eval_set = eval_set_carry,
                    verbose=False)
            importances_carry = pd.DataFrame(zip(xgbr_out_carry.feature_importances_, xgbr_out_carry.get_booster().feature_names))
            importances = importances.merge(importances_carry, left_index = True, how = 'outer', right_index =True)
            score_carry.append(list(xgbr_out_carry.evals_result()["validation_0"].values())[-1][-1])
        print("Importances Extracted")
        print(score_carry)
        importances["average"] = importances.mean(axis = 1)
        importances = importances.set_index(1)
        reduced = iterative_remove(importances,reduced)
        print("DataFrame Reduced",reduced.shape[1])
        
        importances_dict[len(importances)] = importances
        scores[len(importances)] = np.mean(score_carry)
        dfs[len(importances)] = reduced
        
    return  scores, dfs

In [5]:
df = pd.read_pickle("defect_df.p")
df = df[df["defect_formation_energy"] > 0]
y = df["defect_formation_energy"]

In [7]:
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)
scores,dfs = iterative_train(df_soap, kf, thresh = 100, params = {
  "alpha": 2.168396279258295e-05,
  "colsample_bytree": 0.6373043942650429,
  "gamma": 0.0,
  "lambda": 0.00023400919471541485,
  "learning_rate": 0.12402885823965334,
  "max_depth": 5,
  "min_child_weight": 2,
  "subsample": 0.9355880227417938
})

/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


769


/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/tmp/ipykernel_111/775721422.py:62: FutureWarning: Passing 'suffixes' which cause duplicate

Importances Extracted
[0.621212, 0.650066, 0.617006, 0.658439, 0.635146]
DataFrame Reduced 692
692


/tmp/ipykernel_111/775721422.py:66: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  importances["average"] = importances.mean(axis = 1)
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: Futur

...

/tmp/ipykernel_111/775721422.py:66: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  importances["average"] = importances.mean(axis = 1)
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: Futur

Importances Extracted
[0.6255, 0.678876, 0.639452, 0.685512, 0.641787]
DataFrame Reduced 103
103


/tmp/ipykernel_111/775721422.py:66: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  importances["average"] = importances.mean(axis = 1)
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: Futur

Importances Extracted
[0.650499, 0.671734, 0.635437, 0.663854, 0.646715]
DataFrame Reduced 93


/tmp/ipykernel_111/775721422.py:66: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  importances["average"] = importances.mean(axis = 1)


In [3]:
df_soap

,mean SOAP_0,mean SOAP_1,std_dev SOAP_1,std_dev SOAP_18,std_dev SOAP_20,mean SOAP_21,mean SOAP_22,mean SOAP_26,std_dev SOAP_26,mean SOAP_31,...,mean SOAP_6510,mean SOAP_6511,mean SOAP_6515,mean SOAP_6520,mean SOAP_6532,mean SOAP_6536,mean SOAP_6541,mean SOAP_6553,mean SOAP_6574,mean SOAP_6595
1917,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1827,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
352,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1868,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1095,0.039155,-0.148620,0.036540,73.019879,51.467169,0.004901,-0.008617,0.090946,0.057189,-0.161291,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1130,0.019325,-0.057831,0.003446,85.896909,34.129362,0.009374,-0.016154,0.262593,0.264668,-0.452513,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1294,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
def corr_reduction(corr, thresh, data):    
    columns = np.full((corr.shape[0],), True, dtype=bool)
    for i in range(corr.shape[0]):
        for j in range(i+1, corr.shape[0]):
            if corr.iloc[i,j] >= thresh:
                if columns[j]:
                    columns[j] = False
    selected_columns = data.columns[columns]
    data = data[selected_columns]
    return data

# SOAP + CM

In [32]:
df_cm = pd.read_pickle("defect_df_cm_5")
df_cm_reduced = df_cm[df_cm["defect_formation_energy"] > 0]

In [34]:
df_cm_reduced = df_cm_reduced.drop(["name","formula","mp_id", "host_structure", "defect_index", "defect_structure_relaxed", "defect_structure_original","defect_formation_energy"],axis=1)

df_cm_reduced = corr_reduction(df_cm_reduced.corr(),0.9, df_cm_reduced)

In [35]:
df_soap_cm = df_soap_reduced20.join(df_cm_reduced)

In [36]:
df_soap_cm

,mean SOAP_5440,mean SOAP_3088,mean SOAP_5459,std_dev SOAP_2096,std_dev SOAP_1343,mean SOAP_5438,mean SOAP_84,mean SOAP_3071,std_dev SOAP_1766,mean SOAP_1676,...,coulomb matrix eig 213,coulomb matrix eig 219,coulomb matrix eig 223,coulomb matrix eig 227,coulomb matrix eig 228,coulomb matrix eig 230,coulomb matrix eig 246,coulomb matrix eig 250,coulomb matrix eig 295,coulomb matrix eig 322
1917,0.000000,0.0,0.000000,0.0,0.000000,-0.000476,0.000000,0.000000,0.0,0.0,...,66.692702,66.486861,65.377399,65.022843,65.059034,52.259756,63.430295,63.920303,60.184949,55.660664
1827,-0.000255,0.0,-0.028869,0.0,0.000000,0.000000,0.000000,0.001655,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
352,-0.000036,0.0,-0.003998,0.0,-0.016126,0.000000,0.000000,0.000000,0.0,0.0,...,62.721210,61.853641,61.611376,61.353864,63.595772,54.432603,56.798344,56.430765,55.824070,55.539354
91,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,61.622248,60.849285,54.665089,53.721444,53.721444,53.721444,59.277028,53.426846,57.618264,0.000000
1868,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000032,0.0,0.0,...,62.719090,64.295155,56.050513,63.426996,63.426996,60.940593,56.709062,58.554125,56.808192,57.125968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,65.528064,69.032064,53.127010,66.378294,65.360334,63.659429,63.059285,62.195176,60.911179,58.350280
1095,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000352,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1130,0.000000,0.0,0.000000,0.0,0.000000,0.000000,-0.000808,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1294,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [49]:
from sklearn.ensemble import RandomForestRegressor
rf_soap_cv = RandomForestRegressor()
from xgboost import XGBRegressor
xgbr = XGBRegressor()

In [197]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

In [38]:
out_soap_cm = cross_validate(rf_soap_cv, df_soap_cm, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True,error_score = 'raise')

In [50]:
out_cm_gbr = cross_validate(xgbr, df_soap_cm, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True,error_score = 'raise')

/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: Future

In [39]:
out_soap_cm['test_score'].mean()

-0.7115851561313867

In [51]:
out_cm_gbr['test_score'].mean()

-0.6359216104559904

# SOAP + SCM

In [40]:
df_scm = pd.read_pickle("defect_df_scm_5")
df_scm_reduced = df_scm[df_scm["defect_formation_energy"] > 0]

In [41]:
df_scm_reduced = df_scm_reduced.drop(["name","formula","mp_id", "host_structure", "defect_index", "defect_structure_relaxed", "defect_structure_original","defect_formation_energy"],axis=1)

df_scm_reduced = corr_reduction(df_scm_reduced.corr(),0.9, df_scm_reduced)

In [42]:
df_soap_scm = df_soap_reduced20.join(df_scm_reduced)

In [46]:
df_soap_scm_reduced = corr_reduction(df_soap_scm.corr(),0.9,df_soap_scm)

In [43]:
df_soap_scm.head()

,mean SOAP_5440,mean SOAP_3088,mean SOAP_5459,std_dev SOAP_2096,std_dev SOAP_1343,mean SOAP_5438,mean SOAP_84,mean SOAP_3071,std_dev SOAP_1766,mean SOAP_1676,...,sine coulomb matrix eig 144,sine coulomb matrix eig 181,sine coulomb matrix eig 192,sine coulomb matrix eig 193,sine coulomb matrix eig 216,sine coulomb matrix eig 217,sine coulomb matrix eig 223,sine coulomb matrix eig 293,sine coulomb matrix eig 335,sine coulomb matrix eig 401
1917,0.000000,0.0,0.000000,0.0,0.000000,-0.000476,0.0,0.000000,0.0,0.0,...,1644.805984,90.126673,80.986545,80.639932,41.292350,41.336154,42.792877,44.301388,50.295898,53.749478
1827,-0.000255,0.0,-0.028869,0.0,0.000000,0.000000,0.0,0.001655,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
352,-0.000036,0.0,-0.003998,0.0,-0.016126,0.000000,0.0,0.000000,0.0,0.0,...,30.567663,34.606064,75.603930,75.170205,69.783327,75.826760,42.394642,61.545370,53.717163,0.000000
91,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,-13.431210,-13.115136,48.319659,48.412045,64.564599,64.267563,71.139325,59.899083,0.000000,0.000000
1868,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000032,0.0,0.0,...,137.943710,72.385616,71.138847,71.166065,51.514002,66.325768,64.909046,56.641210,0.000000,0.000000


In [44]:
# rf_soap_scm = RandomForestRegressor(**{'bootstrap': True,
#  'max_depth': None,
#  'max_features': 3,
#  'min_samples_leaf': 1,
#  'min_samples_split': 2,
#  'n_estimators': 500})
out_soap_scm = cross_validate(rf_soap_cv, df_soap_scm, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True)


In [53]:
out_scm_gbr = cross_validate(xgbr, df_soap_scm, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True,error_score = 'raise')

/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: Future

In [45]:
out_soap_scm['test_score'].mean()

-0.6996356362020176

In [54]:
out_scm_gbr['test_score'].mean()

-0.6450786448368115

In [122]:
from sklearn.model_selection import GridSearchCV
kf3 = KFold(n_splits = 3, shuffle = True, random_state = 42)

grid = {
    'learning_rate' : [0.1,0.2,0.5,1.0],
    'n_estimators' : [100,200,500],
    'subsample' : [0.5,1.0,2.0],
    'max_depth': [None,1,3,4],
    'max_features': [2, 3],
    'min_samples_leaf': [1,2,3,4],
    'min_samples_split': [2,4,6]
}

grid2 = {
    'learning_rate' : [0.15,0.20,0.15],
    'n_estimators' : [400,500,750],
    'subsample' : [0.9,1.0,1.1],
    'max_depth': [5,6,7],
    'max_features': [4,5,6],
    'min_samples_leaf': [3,4,5,6],
    'min_samples_split': [6,8,10]
}
grid_search_gbr_scm = GridSearchCV(estimator = gbr_soap, param_grid =grid2, 
                          cv = kf3, n_jobs = -1, verbose = 1,scoring = 'neg_root_mean_squared_error')


In [123]:
grid_search_gbr_scm.fit(df_soap_scm,y)


Fitting 3 folds for each of 2916 candidates, totalling 8748 fits


/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
2916 fits failed out of a total of 8748.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2916 fits failed with the following error:
Traceback (most recent call last):
  File "/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/sklearn/ensemble/_gb.py", line 525, in fit
    self._check_params()
  File "/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/sklearn/ensemble/_gb.py", line 317, in _check_para

GridSearchCV(cv=KFold(n_splits=3, random_state=42, shuffle=True),
             estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.15, 0.2, 0.15],
                         'max_depth': [5, 6, 7], 'max_features': [4, 5, 6],
                         'min_samples_leaf': [3, 4, 5, 6],
                         'min_samples_split': [6, 8, 10],
                         'n_estimators': [400, 500, 750],
                         'subsample': [0.9, 1.0, 1.1]},
             scoring='neg_root_mean_squared_error', verbose=1)

In [128]:
gbr_params = {'learning_rate': 0.15,
 'max_depth': 5,
 'max_features': 4,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 400,
 'subsample': 1.0}

SyntaxError: invalid syntax (2447153545.py, line 1)

In [124]:
grid_search_gbr_scm.best_score_

-0.6402159029022508

In [129]:
gbr_scm = GradientBoostingRegressor(**gbr_params)

In [130]:
out_soap_scm = cross_validate(gbr_scm, df_soap_scm, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True)


In [132]:
out_soap_scm['test_score']

array([-0.62881076, -0.65815203, -0.65262021, -0.66776824, -0.64349931])

# SOAP + SCM + Density

In [55]:
df_d = pd.read_pickle("df_d_mpe_gs.pkl")
df_d_reduced = df_d[df_d["defect_formation_energy"] > 0]

In [58]:
df_d_reduced = df_d_reduced.drop(["name","formula","mp_id", "host_structure", "defect_index", "defect_structure_relaxed", "defect_structure_original","defect_formation_energy"],axis=1)

# df_cm_reduced = corr_reduction(df_cm_reduced.corr(),0.9, df_cm_reduced)

In [59]:
df_soap_scd = df_soap_scm.join(df_d_reduced)

In [64]:
df_soap_scd = df_soap_scd.drop(["crystal_system"], axis = 1)

In [65]:
rf_soap_scmd = RandomForestRegressor(**{'bootstrap': True,
 'max_depth': None,
 'max_features': 3,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 500})

rf_soap_scd = RandomForestRegressor()
out_soap_scd = cross_validate(rf_soap_scd, df_soap_scd, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True)


In [66]:
out_soap_scd['test_score'].mean()

-0.7045352585928455

In [67]:
test = pd.read_pickle("rf_soap.pkl")

In [68]:
test

RandomForestRegressor(random_state=42)

In [141]:
from sklearn.model_selection import GridSearchCV
kf3 = KFold(n_splits = 3, shuffle = True, random_state = 42)

grid = {
    'learning_rate' : [0.1,0.2,0.5,1.0],
    'n_estimators' : [100,200,500],
    'subsample' : [0.5,1.0,2.0],
    'max_depth': [None,1,3,4],
    'max_features': [2, 3],
    'min_samples_leaf': [1,2,3,4],
    'min_samples_split': [2,4,6]
}

grid2 = {
    'learning_rate' : [0.15,0.20,0.15],
    'n_estimators' : [400,500,750],
    'subsample' : [0.9,1.0,1.1],
    'max_depth': [5,6,7],
    'max_features': [4,5,6],
    'min_samples_leaf': [3,4,5,6],
    'min_samples_split': [6,8,10]
}
grid_search_gbr_scm = GridSearchCV(estimator = gbr_soap, param_grid =grid2, 
                          cv = kf3, n_jobs = -1, verbose = 1,scoring = 'neg_root_mean_squared_error')


In [142]:
grid_search_gbr_scm.fit(df_soap_scd,y)


Fitting 3 folds for each of 2916 candidates, totalling 8748 fits


/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
2916 fits failed out of a total of 8748.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2916 fits failed with the following error:
Traceback (most recent call last):
  File "/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/sklearn/ensemble/_gb.py", line 525, in fit
    self._check_params()
  File "/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/sklearn/ensemble/_gb.py", line 317, in _check_para

GridSearchCV(cv=KFold(n_splits=3, random_state=42, shuffle=True),
             estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.15, 0.2, 0.15],
                         'max_depth': [5, 6, 7], 'max_features': [4, 5, 6],
                         'min_samples_leaf': [3, 4, 5, 6],
                         'min_samples_split': [6, 8, 10],
                         'n_estimators': [400, 500, 750],
                         'subsample': [0.9, 1.0, 1.1]},
             scoring='neg_root_mean_squared_error', verbose=1)

In [240]:
gbr_params = {'learning_rate': 0.15,
 'max_depth': 5,
 'max_features': 4,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 400,
 'subsample': 1.0}

In [241]:
gbr = GradientBoostingRegressor(**gbr_params)

In [147]:
grid_search_gbr_scm.best_params_

{'learning_rate': 0.15,
 'max_depth': 5,
 'max_features': 5,
 'min_samples_leaf': 4,
 'min_samples_split': 6,
 'n_estimators': 750,
 'subsample': 1.0}

In [144]:
gbr_scm = GradientBoostingRegressor()

In [145]:
out_soap_scm = cross_validate(gbr_scm, df_soap_scd, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True)


In [146]:
out_soap_scm['test_score']

array([-0.75167658, -0.77625191, -0.72879998, -0.78317147, -0.74146849])

# SOAP + CrystalNN

In [69]:
df_c = pd.read_pickle("defect_df_cnn_5")
df_c_reduced = df_c[df_c["defect_formation_energy"] > 0]

In [70]:
df_c_reduced = df_c_reduced.drop(["name","formula","mp_id", "host_structure", "defect_index", "defect_structure_relaxed", "defect_structure_original","defect_formation_energy"],axis=1)

df_c_reduced = corr_reduction(df_c_reduced.corr(),0.9, df_c_reduced)

In [71]:
df_soap_c = df_soap_reduced20.join(df_c_reduced)

In [111]:
df_soap_c.head()

,mean SOAP_3487,mean SOAP_483,mean SOAP_472,mean SOAP_147,mean SOAP_1767,mean SOAP_463,mean SOAP_488,mean SOAP_484,mean SOAP_1639,mean SOAP_493,...,mean wt CN_16,mean wt CN_17,mean wt CN_18,mean wt CN_19,mean wt CN_20,mean wt CN_21,mean wt CN_22,mean wt CN_23,std_dev wt CN_23,mean wt CN_24
1917,0.0,0.004263,-0.038738,0.0,0.0,-0.001378,-0.009364,-0.005291,0.0,0.008827,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1827,0.0,0.008596,-0.134057,0.0,0.0,-0.008352,0.085217,-0.011435,0.0,-0.130709,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
352,0.0,0.012025,0.002674,0.0,0.0,-0.000822,0.093787,-0.015314,0.0,-0.120356,...,0.000007,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,0.0,0.027876,-0.054597,0.0,0.0,-0.002646,0.263299,-0.037189,0.0,-0.354345,...,0.006133,0.000245,0.000009,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1868,0.0,0.012995,-0.051404,0.0,0.0,-0.002645,0.096816,-0.016364,0.0,-0.136496,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
out_soap_c = cross_validate(rf_soap_cv, df_soap_c, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True)


In [73]:
out_soap_c['test_score'].mean()

-0.676393979961268

In [74]:
out_scm_gbr = cross_validate(xgbr, df_soap_c, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True,error_score = 'raise')

/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: Future

In [75]:
out_scm_gbr['test_score'].mean()

-0.6033802785851948

In [83]:
import os
os.environ["SIGOPT_API_TOKEN"] = API_Key

In [116]:
import xgboost
from sklearn.model_selection import train_test_split

X_train, X_test, y_train,y_test = train_test_split(df_soap_c, y, test_size = 0.15, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.20, random_state = 1337)
dtrain = xgboost.DMatrix(data=X_train, label=y_train, feature_names=df_soap_c.columns)
dtest = xgboost.DMatrix(data=X_test, label=y_test, feature_names=df_soap_c.columns)      
dval = xgboost.DMatrix(data=X_val, label=y_val, feature_names=df_soap_c.columns)
dset = xgboost.DMatrix(data = df_soap_c, label = y, feature_names = df_soap_c.columns)

In [99]:
import sigopt
import sigopt.xgboost
arguments = {
    "dtrain": dtrain,
    "params": {
        "objective": "reg:gamma"
    },
     "evals": [(dtest, "Test"), (dval, "Validation")],
     "experiment_config": {"name": "Automatic XGBoost Tuning - CrystalNN, Validation", "budget": 100}
}
experiment = sigopt.xgboost.experiment(**arguments)

In [111]:
experiment.get_best_runs()
	0	16	0	180
0.3077903126305188	0	8	2.4499610780673606	200

<generator object Pagination.iterate_pages at 0x7f123d4a0900>

In [123]:
params = {
    "eta":0.4193806487882235,
    "gamma":0,
    "max_depth":16,
    "min_child_weight":0}

params2 = {
    "eta":0.3077903126305188,
    "gamma":0,
    "max_depth":8,
    "min_child_weight":2.4499610780673606}

In [127]:
out_xgbr_c = xgboost.cv(params, dset, num_boost_round =200, folds = kf, metrics = 'rmse', as_pandas = True, seed = 42, shuffle = True)


In [151]:
out_xgbr_c.iloc[-1]["test-rmse-mean"]

0.6709695999999999

In [203]:
import xgboost as xgb
import warnings
from sklearn.model_selection import cross_val_score
kf3 = KFold(n_splits = 3, shuffle = True, random_state = 42)

warnings.filterwarnings("ignore") 
def create_model(assignments):
    model = xgb.XGBRegressor(
                                n_estimators     = assignments['n_estimators'],
                                min_child_weight = assignments['min_child_weight'],
                                max_depth        = assignments['max_depth'],
                                gamma            = assignments['gamma'],
                                subsample        = assignments['subsample'],
                                colsample_bytree = assignments['colsample_bytree'],
                                reg_lambda       = assignments['lambda'],
                                reg_alpha        = assignments['alpha'],
                                learning_rate    = assignments['log_learning_rate'],
                                n_jobs           = 4
                            )
    return model
def evaluate_model(assignments):
    model = create_model(assignments)
    out_soap_c = cross_validate(rf_soap_cv, df_soap_c, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True)
    scores = cross_val_score(model, df_soap_c, y, scoring = 'neg_root_mean_squared_error', cv = kf3)
    rmse = scores.mean()
    
    return [ dict(name="RMSE", value = rmse)]
    


In [205]:
from sigopt import Connection
 
# put your token here
api_token = API_Key
 
conn = Connection(client_token=api_token)

In [209]:
experiment = conn.experiments().create(
    
    name="SOAP+CNN - Vanilla SigOpt",
 
    parameters=[
        dict(name="n_estimators", bounds=dict(min=50,max=350), type="int"),
        dict(name="min_child_weight", bounds=dict(min=2,max=15), type="int"),
        dict(name="max_depth", bounds=dict(min=3,max=10), type="int"),
        dict(name="gamma", bounds=dict(min=0,max=5), type="double"),
        dict(name="subsample", bounds=dict(min=0.5,max=1), type="double"),
        dict(name="colsample_bytree", bounds=dict(min=0.5,max=1), type="double"),
        dict(name="lambda", bounds=dict(min=0.00001, max=1), type="double", transformation="log"),
        dict(name="alpha", bounds=dict(min=0.00001, max=1), type="double", transformation="log"),
        dict(name="log_learning_rate", bounds=dict(min=0.00001 ,max=1), type="double", transformation="log")
        ],
 
    metrics=[
        dict(name="RMSE", objective="maximize", strategy="optimize")
        ],
 
    observation_budget = 120,
)
 
print("Explore your experiment: https://app.sigopt.com/experiment/" + experiment.id + "/analysis")

Explore your experiment: https://app.sigopt.com/experiment/486715/analysis


In [210]:
for _ in range(experiment.observation_budget):
    suggestion = conn.experiments(experiment.id).suggestions().create()
    assignments = suggestion.assignments
    value_dicts = evaluate_model(assignments)
 
    conn.experiments(experiment.id).observations().create(
        suggestion=suggestion.id,
        values=value_dicts
    )
    
    #update experiment object
    experiment = conn.experiments(experiment.id).fetch()
 
assignments = conn.experiments(experiment.id).best_assignments().fetch().data[0].assignments  
 
print("BEST ASSIGNMENTS FOUND: \n", assignments)

BEST ASSIGNMENTS FOUND: 
 Assignments({
  "alpha": 1.3071262158536206e-05,
  "colsample_bytree": 0.5887060253641718,
  "gamma": 0.0,
  "lambda": 6.845770146631498e-05,
  "log_learning_rate": 0.1580116151601096,
  "max_depth": 6,
  "min_child_weight": 2,
  "n_estimators": 239,
  "subsample": 0.887616355804612
})


In [198]:
folds = [next(kf.split(df_soap_c)) for i in range(5)]

In [202]:
folds[4][1]


array([  15,   23,   29,   30,   32,   43,   44,   49,   51,   56,   59,
         65,   67,   69,   70,   73,   76,   78,   99,  100,  101,  107,
        109,  115,  123,  124,  128,  135,  141,  162,  163,  168,  170,
        173,  175,  184,  185,  188,  198,  199,  203,  210,  212,  220,
        226,  231,  237,  239,  240,  244,  247,  251,  259,  261,  270,
        271,  274,  275,  289,  297,  298,  300,  303,  316,  324,  331,
        332,  339,  342,  344,  350,  351,  352,  353,  361,  366,  367,
        371,  374,  381,  382,  383,  394,  405,  408,  411,  413,  415,
        416,  420,  425,  426,  427,  429,  432,  438,  439,  451,  453,
        462,  464,  478,  479,  482,  483,  485,  486,  495,  506,  513,
        514,  526,  529,  530,  532,  534,  543,  548,  552,  555,  561,
        566,  567,  575,  576,  583,  589,  591,  598,  599,  602,  607,
        610,  619,  628,  629,  631,  634,  637,  643,  650,  651,  654,
        668,  669,  670,  674,  677,  678,  682,  6

In [171]:
X_train1

,mean SOAP_5440,mean SOAP_3088,mean SOAP_5459,std_dev SOAP_2096,std_dev SOAP_1343,mean SOAP_5438,mean SOAP_84,mean SOAP_3071,std_dev SOAP_1766,mean SOAP_1676,...,mean wt CN_16,mean wt CN_17,mean wt CN_18,mean wt CN_19,mean wt CN_20,mean wt CN_21,mean wt CN_22,mean wt CN_23,std_dev wt CN_23,mean wt CN_24
1917,0.000000,0.0,0.000000,0.0,0.000000,-0.000476,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
352,-0.000036,0.0,-0.003998,0.0,-0.016126,0.000000,0.000000,0.000000,0.0,0.0,...,0.000007,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
91,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.006133,0.000245,0.000009,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1868,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000032,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1508,0.000000,0.0,0.000000,0.0,0.000000,0.000000,-0.000007,0.000000,0.0,0.0,...,0.001983,0.001555,0.000319,0.001485,0.002435,0.000465,0.001537,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1638,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000015,0.000414,0.000021,0.000405,0.000009,0.000000,0.000000,0.0,0.0,0.0
1130,0.000000,0.0,0.000000,0.0,0.000000,0.000000,-0.000808,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1294,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.008050,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0


# SOAP + Minor Featurisation

In [76]:
df_soap_d = df_soap_reduced20.join(df_d_reduced)

In [79]:
df_soap_d = df_soap_d.drop(["crystal_system"], axis = 1)

In [80]:
df_soap_d.head()

,mean SOAP_5440,mean SOAP_3088,mean SOAP_5459,std_dev SOAP_2096,std_dev SOAP_1343,mean SOAP_5438,mean SOAP_84,mean SOAP_3071,std_dev SOAP_1766,mean SOAP_1676,...,mean SOAP_1030,std_dev SOAP_442,std_dev SOAP_2648,density,vpa,packing fraction,max packing efficiency,spacegroup_num,crystal_system_int,is_centrosymmetric
1917,0.000000,0.0,0.000000,0.0,0.000000,-0.000476,0.0,0.000000,0.0,0.0,...,0.0,0.000332,0.0,7.340130,11.959952,0.410482,0.300958,1,7,False
1827,-0.000255,0.0,-0.028869,0.0,0.000000,0.000000,0.0,0.001655,0.0,0.0,...,0.0,-0.000389,0.0,5.662070,16.106118,0.733155,0.311252,1,7,False
352,-0.000036,0.0,-0.003998,0.0,-0.016126,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000515,0.0,5.551771,11.513054,0.685517,0.379453,1,7,False
91,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000332,0.0,5.085798,12.389093,0.794698,0.277284,8,6,False
1868,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000032,0.0,0.0,...,0.0,0.000033,0.0,6.587255,15.571968,0.641528,0.265092,1,7,False


In [81]:
out_soap_d = cross_validate(rf_soap_cv, df_soap_d, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True)


In [82]:
out_soap_d['test_score'].mean()

-0.6960514955287156